# IMPORT FILES & LIBRARIES

In [1]:
# PyTorch for deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F

# PyTorch Geometric for graph neural networks
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

# Data visualization
import matplotlib.pyplot as plt

# Progress bar for training
from tqdm import tqdm

# Spatial distance computations
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors, BallTree
from sklearn.metrics.pairwise import haversine_distances

# HTTP requests
import requests

# import config file
from config import Config

# STEP 1A: CREATE GRAPH

## 1A.1 CREATE NODES

In [2]:
def create_nodes(meta_df, station = 'freeway_id'):
    """
    Create a list of unique nodes (stations) from the metadata DataFrame and generate mappings between node IDs and indices.

    Parameters:
    - meta_df: DataFrame containing metadata with node (station) identifiers.
    - station: Column name in the DataFrame that contains the node identifiers (default is 'freeway_id').

    Returns:
    - nodes: List of unique node identifiers.
    - node_index_map: Dictionary mapping node identifiers to their corresponding index.
    - index_node_map: Dictionary mapping index to node identifiers.
    """

    # Get the nodes list i.e. the stations
    meta_df = meta_df.sort_values(by=station, ascending=True)
    nodes = list(meta_df[station])
    nodes = sorted(list(set(nodes)))       

    # map nodes to index: create node index map & index node map
    node_index_map = {node: i for i, node in enumerate(nodes)}
    index_node_map = {i: node for i, node in enumerate(nodes)}

    print (len(nodes), 'Nodes Created!')

    return nodes, node_index_map, index_node_map

## 1A.2 CREATE EDGES & EDGE ATTRIBUTES 

1. Identify the Euclidean distance between the stations. 
2. Map only those that are within threshold radius miles.
3. This makes it computationally efficient to run OSRM API only for the potential mappings.
4. Map only those stations (edges) that have a driving distance less than threshold radius miles.
5. There are other conditions too like lane type should match.
6. After mapping, edge attributes are created using below logic.
7. One attribute is the normalised percentage acutal distance between the stations (threshold radius-distance)/threshold radius.
8. second attribute is the chance of coming in the road i.e. the route possibility. if same highway and direction then 1 else 0.

### Get Actual Driving Distance (Open Street Map API)

In [3]:
def get_driving_distance_osm(start_lat, start_lon, end_lat, end_lon):
    """
    Function to get the driving distance between two geographic points using OpenStreetMap (OSM) data.
    
    Parameters:
    - start_lat: Latitude of the starting point.
    - start_lon: Longitude of the starting point.
    - end_lat: Latitude of the destination point.
    - end_lon: Longitude of the destination point.

    Returns:
    - Distance in miles between the start and end points.
    """
    
    base_url = Config.osrm_path # Base URL for the OSRM (Open Source Routing Machine) API
    url = f"{base_url}{start_lon},{start_lat};{end_lon},{end_lat}"
    params = {
        "overview": "false", # Minimize the amount of returned data
        "alternatives": "false",  # Do not provide alternative routes
    }
    
    response = requests.get(url, params=params) # Send the request to the OSRM API
    data = response.json()
    
    if data["code"] == "Ok":
        distance = data["routes"][0]["distance"]  # Extract the driving distance in meters
        return distance / 1609  # Convert to miles
    else:
        return None

### Create Edges and Edge Attributes

In [4]:
def create_edge_and_attributes(meta_df, radius_miles=1):
    """
    Function to create edges and edge attributes based on spatial proximity and route characteristics.

    Parameters:
    - meta_df: DataFrame containing metadata about the locations (e.g., stations).
    - radius_miles: The radius within which to consider points as neighbors (default is 1 mile).

    Returns:
    - edges: A tensor containing pairs of indices that define the edges.
    - edge_attributes: A tensor containing attributes for each edge (distance and route similarity).
    """
    
    edges = []
    edge_attributes = []
    
    # Extract coordinates and convert to radians
    coords = np.radians(meta_df[['latitude', 'longitude']].values)
    
    # Create a BallTree for efficient nearest neighbor queries
    tree = BallTree(coords, metric='haversine')
    
    # Convert the radius from miles to radians
    radius_radians = (radius_miles+0.25) / 3959.0
    
    # Query the tree for all points within the radius
    # prints only the indices within the range of distance
    indices, distances = tree.query_radius(coords, r=radius_radians, return_distance=True)
    
    
    # Create adjacency matrix wrt distance and route
    n = len(meta_df)
    distance_matrix = np.zeros((n, n), dtype=float)
    route_matrix = np.zeros((n, n), dtype=float)

    # Loop over each point's neighbors and distances
    for i, (idx, dist) in tqdm(enumerate(zip(indices, distances)), total=len(distances), desc="Creating Edges & Edge Attributes", bar_format="{l_bar}{r_bar}"):  
        # 0-1 distance (towards 1 closer distance)
        # Calculate a weighted distance score for neighbors (closer distance = higher score)
        distance_matrix[i, idx] = np.where(dist>0,(radius_miles-(dist * 3959.0))/radius_miles,0)
        
        # Create a copy of the meta_df to find neighbors' attributes
        meta_df_cpy = meta_df.copy().reset_index()
        meta_df_cpy['distance'] = distance_matrix[i]

        # Filter out neighbors with non-zero distances and retrieve relevant columns
        neighbors = meta_df_cpy.loc[distance_matrix[i] > 0, ['freeway_id', 'freeway_direction','highway','type','distance', 'latitude', 'longitude']]

        # Iterate through the neighbors to evaluate and assign edge attributes
        for j, row in neighbors.iterrows():

            # Only consider neighbors of the same lane type for edge creation
            if meta_df.iloc[i].type != row['type']:
                distance_matrix[i, j] = 0
            else:
                # Calculate the actual driving distance using OpenStreetMap (OSM) data
                actual_distance = get_driving_distance_osm(row['latitude'], row['longitude'], meta_df.iloc[i].latitude, meta_df.iloc[i].longitude)
                # If the actual distance exceeds the radius, ignore this neighbor
                if actual_distance > radius_miles:
                    distance_matrix[i, j] = 0    
                else:
                    # Assign the actual driving distance to the distance matrix
                    distance_matrix[i, j] = np.round(actual_distance,2)
                    # Check if same road, and assign route matrix accordingly
                    route_matrix[i, j] = 1 if (meta_df.iloc[i].freeway_direction == row['freeway_direction']) and (meta_df.iloc[i].highway == row['highway']) and (meta_df.iloc[i].type == row['type']) else 0 
                    edges.append((i, j))
                    # Append the edge and its attributes to their respective lists
                    edge_attributes.append((distance_matrix[i, j], route_matrix[i, j]))
                    
    # Remove self-loops
    np.fill_diagonal(distance_matrix, 0.0)
    np.fill_diagonal(route_matrix, 0.0)
    edges = torch.LongTensor(edges).t().contiguous()
    edge_attributes = torch.tensor(edge_attributes, dtype=torch.float32)
    print ('Edges & Edge Attributes Created!!')
    
    return edges, edge_attributes

## 1A.3 CREATE GRAPH WITH NODE FEATURES

1. 1 graph for each timestep (each timestep refers to the reference timestep which means that we are currently at that instant and we know the truth (traffic flow) only till that point).
2. 41 nodes in each graph with same edges and edge attributes.
3. For each timestep, both x (node features) and y (target) are pointed based on the inputs passed.
4. The inputs independent_var tell the node features and dependent_var tells about the target. 
5. The inputs x_ts_start, x_ts_end, x_ts_step informs based on the current timestep to focus on which past timestep wrt to the current timestep.
6. The inputs y_ts_start, y_ts_end, y_ts_step informs based on the current timestep to focus on which future timestep to predict wrt to the current timestep.

In [5]:
# Create graph for each timestamp
def create_graphs(df, independent_var, dependent_var, edge_index, edge_attributes, x_ts_start = -7, x_ts_end = 5, x_ts_step=1, y_ts_start=1, y_ts_end=4, y_ts_step=1, data_interval_mins=5):

    """
    Function to create graphs based on timestamps from the given DataFrame. 
    The function generates graph data for each unique timestamp in the DataFrame.

    Parameters:
    - df: DataFrame containing the data with timestamps.
    - independent_var: List of column names to be used as independent variables.
    - dependent_var: List of column names to be used as dependent variables.
    - edge_index: Edge indices for graph construction.
    - edge_attributes: Attributes for the edges in the graph.
    - x_ts_start: Start time step (in terms of intervals) for the independent variable window.
    - x_ts_end: End time step (in terms of intervals) for the independent variable window.
    - x_ts_step: Step size (in terms of intervals) for sampling within the independent variable window.
    - y_ts_start: Start time step (in terms of intervals) for the dependent variable window.
    - y_ts_end: End time step (in terms of intervals) for the dependent variable window.
    - y_ts_step: Step size (in terms of intervals) for sampling within the dependent variable window.
    - data_interval_mins: The interval (in minutes) between timestamps in the data.

    Returns:
    - graphs: A list of graph objects created for each unique timestamp.
    - timestamp_sequences: A list of timestamps corresponding to each graph.
    """
    
    # Store the graphs
    graphs = []
    timestamp_sequences = []

    # Convert 'iso_timestamp' column to numpy array of datetime64 type
    timestamps = df['iso_timestamp'].values
    timestamps = np.array(timestamps, dtype='datetime64')
    
    # Convert time steps into timedelta (in nanoseconds)
    x_ts_step = np.int64(np.timedelta64(x_ts_step*data_interval_mins*60*10**9, 's'))
    y_ts_step = np.int64(np.timedelta64(y_ts_step*data_interval_mins*60*10**9, 's'))
    
    
    for timestamp in tqdm(sorted(df['iso_timestamp'].unique())):
        
        # Define graph
        g = Data()
        g.edge_index = edge_index
        g.edge_attr = edge_attributes
        
        # Current iteration time in nanoseconds
        row_time = np.int64(timestamp.timestamp()*10**9)

        # Define range for dependent variable (y) timestamps
        row_y_ts_start = row_time + np.int64(np.timedelta64(y_ts_start*data_interval_mins*60*10**9, 's'))
        row_y_ts_end = row_time + np.int64(np.timedelta64(y_ts_end*data_interval_mins*60*10**9, 's'))
        row_y_ts_range = list(range(row_y_ts_start, row_y_ts_end + y_ts_step, y_ts_step))
        row_x_ts_range = np.array([])
        
        # Define range for independent variable (x) timestamps for each y_ts in row_y_ts_range
        for y_ts in row_y_ts_range:
            
            temp_x_ts_start = y_ts + np.int64(np.timedelta64(x_ts_start*data_interval_mins*60*10**9, 's'))
            temp_x_ts_end = y_ts + np.int64(np.timedelta64(x_ts_end*data_interval_mins*60*10**9, 's'))
            temp_x_ts_range = np.array(list(range(temp_x_ts_start, temp_x_ts_end + x_ts_step, x_ts_step)))
            
            # Keep only timestamps that are earlier than or equal to the current timestamp
            temp_x_ts_range = temp_x_ts_range[temp_x_ts_range<=row_time]
            row_x_ts_range = np.unique(np.hstack([row_x_ts_range, temp_x_ts_range]))
            

        # Convert to datetime64 and find indices in the original DataFrame
        row_x_ts_range = np.array(row_x_ts_range, dtype='datetime64[ns]')
        row_x_ts_range = list(row_x_ts_range.astype(str))
        row_x_ts_range = np.array(row_x_ts_range, dtype='datetime64')
        row_x_indices = np.where(np.isin(timestamps, row_x_ts_range))[0]
            
        row_y_ts_range = np.array(row_y_ts_range, dtype='datetime64[ns]')
        row_y_ts_range = list(row_y_ts_range.astype(str))
        row_y_ts_range = np.array(row_y_ts_range, dtype='datetime64')
        row_y_indices = np.where(np.isin(timestamps, row_y_ts_range))[0]

        
        # Create pivot table for independent variables (x)
        pivot_df_x = df[['station','iso_timestamp'] + independent_var].copy().loc[sorted(row_x_indices)].pivot(index='station', columns='iso_timestamp')
 
            
        if len(pivot_df_x)>0: 
            
            # Flatten multi-level columns and sort them (traffic features for x timeframe)
            pivot_df_x.columns = [f'{col[1]}_{col[0]}' for col in pivot_df_x.columns]
            sorted_columns = sorted(pivot_df_x.columns)
            pivot_df_x = pivot_df_x[sorted_columns]
            pivot_df_x = pivot_df_x.apply(pd.to_numeric)
            g.x = torch.tensor(pivot_df_x.values, dtype=torch.float)

            # Create pivot table for dependent variables (y)
            pivot_df_y = df[['station','iso_timestamp'] + dependent_var].loc[sorted(row_y_indices)].pivot(index='station', columns='iso_timestamp')

            if len(pivot_df_y.columns)>=len(row_y_ts_range)*len(dependent_var):
                # Flatten multi-level columns and sort them
                pivot_df_y.columns = [f'{col[1]}_{col[0]}' for col in pivot_df_y.columns]
                sorted_columns = sorted(pivot_df_y.columns)
                pivot_df_y = pivot_df_y[sorted_columns]
                pivot_df_y = pivot_df_y.apply(pd.to_numeric)
                g.y = torch.tensor(pivot_df_y.values, dtype=torch.float)
                graphs.append(g)
                
                timestamp_sequences.append(timestamp)
            else:
                timestamp_sequences.append(timestamp)
                

    return graphs, timestamp_sequences

# STEP 1B: CREATE GRIDS

## 1B.1 GRID MAPPING

1. Find the n nearest neigbours whose traffic value will be used to predict

In [6]:
def create_grid_mapping(meta_df, neighbours, node_index_map, lat_range=None, lon_range=None):

    """
    Function to create a grid mapping for stations based on their geographical proximity.
    The mapping is created by first sorting stations by 'highway' and 'freeway_direction', 
    then grouping them and finding nearest neighbors based on geographical distance.

    Parameters:
    - meta_df: DataFrame containing station metadata including 'latitude' and 'longitude'.
    - neighbours: Number of nearest neighbors to include in the grid for each station.
    - node_index_map: Dictionary mapping station IDs to their respective indices.
    - lat_range: Optional, latitude range to filter the stations.
    - lon_range: Optional, longitude range to filter the stations.

    Returns:
    - grid: A sorted list where each entry represents a station and its nearest neighbors,
            with station IDs mapped to their indices.
    """

    # Sort the metadata DataFrame by 'highway' and 'freeway_direction'
    meta_df = meta_df.sort_values(by = ['highway','freeway_direction'])
    grid = []
    
    # Function to map station IDs to their corresponding index using 'node_index_map'
    def map_station_index(x):
        return node_index_map[x]
    
    # Iterate over each unique combination of 'highway' and 'freeway_direction'
    for row, current_data in meta_df[['highway','freeway_direction']].drop_duplicates().iterrows():
        
        # Filter the DataFrame for the current 'highway' and 'freeway_direction'
        temp_station_metadata = meta_df.loc[(meta_df['highway']==current_data['highway']) 
                                                     & (meta_df['freeway_direction']==current_data['freeway_direction'])].copy()

        # Calculate the pairwise haversine distances (geodesic distance) between stations
        distances = haversine_distances(np.radians(temp_station_metadata[['latitude', 'longitude']].values))
        # Fit a nearest neighbors model on the calculated distances
        nbrs = NearestNeighbors(n_neighbors = neighbours+1, algorithm='ball_tree').fit(distances)
        _, indices = nbrs.kneighbors(distances)

        # For each station in the current group
        for i in range(len(temp_station_metadata)):

            # Find the indices of the nearest neighbors, excluding the station itself
            idx = indices[i][indices[i]!=i]
            i_idx =  indices[i][indices[i]==i]

            # Concatenate the station and its neighbors into one row
            grid_row = pd.concat([temp_station_metadata.iloc[i_idx,0], 
                                  temp_station_metadata.iloc[idx,0]])
            # Map station IDs to indices using the 'map_station_index' function and add to grid
            grid.append(np.vectorize(map_station_index)(grid_row))
    
    # Sort the grid based on the first element of each row (the station index)
    grid = sorted(grid, key=lambda x: x[0])

    return grid

## 1B.2 GRID ROW FEATURE MAPPING: MAP NEIGHBOURING DATA

In [7]:
def to_spatial_grid(df, grid_mapping):
    """
    Function to transform the DataFrame into a spatial grid based on the grid mapping.

    Parameters:
    - df: DataFrame containing the data to be transformed into the spatial grid.
    - grid_mapping: The grid mapping created by 'create_grid_mapping' function.

    Returns:
    - torch.tensor: A PyTorch tensor representing the spatial grid, 
                    where each row corresponds to the flattened values of stations and their neighbors.
    """
    
    result = []
    
    # Iterate over each grid row in the grid mapping
    for row in grid_mapping:
        # Flatten the data of the stations in the grid row and append to result list
        result.append(df.iloc[row].values.flatten())

    # Convert the result to a PyTorch tensor
    return torch.tensor(np.array(result), dtype=torch.float)

## 1B.3 CREATE GRID FEATURES DATA

1. 1 grid for each timestep (each timestep refers to the reference timestep which means that we are currently at that instant and we know the truth till that point)
2. 41 rows in each grid, each row representa a station data for that timestep
3. For each timestep, both x (grid features) and y (target) are pointed based on the inputs passed.
4. The inputs independent_var tell the features of both reference station and mapped neighbour stations and dependent_var tells about the target of reference station. 
5. The inputs x_ts_start, x_ts_end, x_ts_step tells based on the current timestep to focus on which past timestep wrt to the current timestep.
6. The inputs y_ts_start, y_ts_end, y_ts_step tells based on the current timestep to focus on which future timestep to predict wrt to the current timestep.

In [8]:
# Create grid for each timestamp
def create_grids(df, independent_var, dependent_var, grid_mapping, x_ts_start = -7, x_ts_end = 5, x_ts_step=1, y_ts_start=1, y_ts_end=4, y_ts_step=1, data_interval_mins=5):
    """
    Function to create spatial grids for each timestamp in the DataFrame.
    The function generates grid data for each unique timestamp, using a given grid mapping
    and time intervals to determine the relevant data points.

    Parameters:
    - df: DataFrame containing the data with timestamps.
    - independent_var: List of column names to be used as independent variables.
    - dependent_var: List of column names to be used as dependent variables.
    - grid_mapping: A mapping that defines the spatial grid structure.
    - x_ts_start: Start time step (in terms of intervals) for the independent variable window.
    - x_ts_end: End time step (in terms of intervals) for the independent variable window.
    - x_ts_step: Step size (in terms of intervals) for sampling within the independent variable window.
    - y_ts_start: Start time step (in terms of intervals) for the dependent variable window.
    - y_ts_end: End time step (in terms of intervals) for the dependent variable window.
    - y_ts_step: Step size (in terms of intervals) for sampling within the dependent variable window.
    - data_interval_mins: The interval (in minutes) between timestamps in the data.

    Returns:
    - grids: A list of grid objects created for each unique timestamp.
    - timestamp_sequences: A list of timestamps corresponding to each grid.
    """
    
    # Store the grids for each timestamp
    grids = []
    timestamp_sequences = []

    # Convert 'iso_timestamp' column to numpy array of datetime64 type
    timestamps = df['iso_timestamp'].values
    timestamps = np.array(timestamps, dtype='datetime64')

    # Convert time steps into timedelta (in nanoseconds)
    x_ts_step = np.int64(np.timedelta64(x_ts_step*data_interval_mins*60*10**9, 's'))
    y_ts_step = np.int64(np.timedelta64(y_ts_step*data_interval_mins*60*10**9, 's'))
    

    # Iterate over each unique timestamp in the DataFrame
    for timestamp in tqdm(sorted(df['iso_timestamp'].unique())):
        
        # Create an empty Data object to store grid data for the current timestamp
        g = Data()
        
        # Current iteration time in nanoseconds
        row_time = np.int64(timestamp.timestamp()*10**9)

        # Define range for dependent variable (y) timestamps
        row_y_ts_start = row_time + np.int64(np.timedelta64(y_ts_start*data_interval_mins*60*10**9, 's'))
        row_y_ts_end = row_time + np.int64(np.timedelta64(y_ts_end*data_interval_mins*60*10**9, 's'))
        row_y_ts_range = list(range(row_y_ts_start, row_y_ts_end + y_ts_step, y_ts_step))
        row_x_ts_range = np.array([])

        
        # Define range for independent variable (x) timestamps for each y_ts in row_y_ts_range
        for y_ts in row_y_ts_range:
            
            temp_x_ts_start = y_ts + np.int64(np.timedelta64(x_ts_start*data_interval_mins*60*10**9, 's'))
            temp_x_ts_end = y_ts + np.int64(np.timedelta64(x_ts_end*data_interval_mins*60*10**9, 's'))
            temp_x_ts_range = np.array(list(range(temp_x_ts_start, temp_x_ts_end + x_ts_step, x_ts_step)))

            # Keep only timestamps that are earlier than or equal to the current timestamp
            temp_x_ts_range = temp_x_ts_range[temp_x_ts_range<=row_time]
            row_x_ts_range = np.unique(np.hstack([row_x_ts_range, temp_x_ts_range]))
            

        # Convert to datetime64 and find indices in the original DataFrame
        row_x_ts_range = np.array(row_x_ts_range, dtype='datetime64[ns]')
        row_x_ts_range = list(row_x_ts_range.astype(str))
        row_x_ts_range = np.array(row_x_ts_range, dtype='datetime64')
        row_x_indices = np.where(np.isin(timestamps, row_x_ts_range))[0]
        
        row_y_ts_range = np.array(row_y_ts_range, dtype='datetime64[ns]')
        row_y_ts_range = list(row_y_ts_range.astype(str))
        row_y_ts_range = np.array(row_y_ts_range, dtype='datetime64')
        row_y_indices = np.where(np.isin(timestamps, row_y_ts_range))[0]

        # Pivot the DataFrame for independent variables (x)
        pivot_df_x = df[['station','iso_timestamp'] + independent_var].copy().loc[sorted(row_x_indices)].pivot(index='station', columns='iso_timestamp')
 
            
        if len(pivot_df_x)>0: 
            
            # Flatten the multi-level columns and sort them (traffic features)
            pivot_df_x.columns = [f'{col[1]}_{col[0]}' for col in pivot_df_x.columns]
            sorted_columns = sorted(pivot_df_x.columns)
            pivot_df_x = pivot_df_x[sorted_columns]
            pivot_df_x = pivot_df_x.apply(pd.to_numeric)

            # Convert the pivot table to a spatial grid using the grid mapping
            g.x = to_spatial_grid(pivot_df_x, grid_mapping)

            # Pivot the DataFrame for dependent variables (y)
            pivot_df_y = df[['station','iso_timestamp'] + dependent_var].loc[sorted(row_y_indices)].pivot(index='station', columns='iso_timestamp')

            # Check if the pivot table has enough columns to match the expected length
            if len(pivot_df_y.columns)>=len(row_y_ts_range)*len(dependent_var):
                # Flatten the multi-level columns and sort them
                pivot_df_y.columns = [f'{col[1]}_{col[0]}' for col in pivot_df_y.columns]
                sorted_columns = sorted(pivot_df_y.columns)
                pivot_df_y = pivot_df_y[sorted_columns]
                pivot_df_y = pivot_df_y.apply(pd.to_numeric)

                # Convert the pivot table to a PyTorch tensor and store it as the target variable
                g.y = torch.tensor(pivot_df_y.values, dtype=torch.float)
                grids.append(g)

                # Append the grid data and corresponding timestamp to the result lists
                timestamp_sequences.append(timestamp)
            else:
                # If the y data is insufficient, still store the timestamp
                timestamp_sequences.append(timestamp)
                
    # Return the list of grids and corresponding timestamps
    return grids, timestamp_sequences


# STEP 2. MATCH INPUT LENGTH

In [9]:
def match_input_dim(data, timestamp):
    """
    Function to filter and retain only those data samples where the input dimensions match
    the desired dimensions, ensuring consistent input size for further processing.

    Parameters:
    - data: List of data objects, each containing features (x).
    - timestamp: List of timestamps corresponding to each data object.

    Returns:
    - data_final: List of filtered data objects with matching input dimensions.
    - timestamp_final: List of corresponding timestamps for the filtered data.
    """
    
    data_final = []
    timestamp_final = []
    
    for i in range(len(data)):

        start_dim = data[i].x.shape[1] # Input dimension of the current data object
        end_dim = data[-1].x.shape[1] # Input dimension of the last data object

        # If the input dimensions match, retain this data object and its timestamp
        if start_dim == end_dim:
            data_final.append(data[i])
            timestamp_final.append(timestamp[i])

    return data_final, timestamp_final

# STEP 3. CREATE MODEL

1. 1st layer can either be CNN/GCN/GAT and can have multiple parallel layers
2. After this, the output of the 1st layer is processed, the embeddings are concatenated
3. Then passed to a Transformer
4. Followed by FC layer

## MODEL ARCHITECTURE

In [10]:
class DualGAT_Trans(nn.Module):
    def __init__(self, model_params):
        """
        Initialize the DualGAT_Trans model, which is a spatial-temporal neural network.
        
        Parameters:
        - model_params: A list of dictionaries containing parameters for each layer in the network.
                        Each dictionary specifies the model type (e.g., CNN, GCN, GAT, Transformer) 
                        and relevant parameters for that layer.
        """
        super(DualGAT_Trans, self).__init__()

        # Initialize a ModuleList to hold all layers in the network
        self.layers = nn.ModuleList()

        # Loop through the provided model parameters to create layers
        for params in model_params:
            layer_no = params.pop('layer_no') # Layer number
            model_type = params.pop('model') # Type of model (e.g., CNN, GCN, GAT, Transformer)

            if layer_no == 1: # First layer can be CNN, GCN, or GAT
                if model_type == 'CNN':
                    self.layers.append(nn.Conv2d(**params))
                elif model_type == 'GCN':
                    self.layers.append(GCNConv(**params))
                elif model_type == 'GAT':
                    self.layers.append(GATConv(**params))
                else:
                    raise ValueError(f"Unknown model type: {model_type}")
            
            else: # Subsequent layers can be Transformer or Linear
                if model_type == 'Transformer':
                    params['batch_first'] = True # Set batch_first to True for transformers
                    self.transformer = nn.Transformer(**params)
                elif model_type == 'Linear':
                    self.linear = nn.Linear(**params)
                else:
                    raise ValueError(f"Unknown model type: {model_type}")

        # Define a max-pooling layer for use with CNNs
        self.pool = nn.MaxPool2d(kernel_size=Config.cnn_pooling_size, stride=Config.cnn_pooling_stride)

    def forward(self, data_list):
        """
        Forward pass for the DualGAT_Trans model.

        Parameters:
        - data_list: A list of input data objects.

        Returns:
        - x: The output of the network after passing through all layers and the final linear transformation.
        """
        
        x_list = []
        batch_size = data_list[0].num_graphs # Number of graphs in the batch

        # Apply the first layer to each data object in data_list
        for i, data in enumerate(data_list):
            x = data.x if hasattr(data, 'x') else data
            edge_index = data.edge_index if hasattr(data, 'edge_index') else None

            # If the layer is a GCN or GAT, apply it to the graph input
            if isinstance(self.layers[i], (GCNConv, GATConv)):
                x = self.layers[i](x, edge_index)

            # If the layer is a CNN, reshape and apply it to the grid input
            elif isinstance(self.layers[i], nn.Conv2d):
                x = torch.reshape(x, (batch_size, -1, x.shape[-1]))
                x = x.unsqueeze(1)  # Add channel dimension
                x = F.relu(self.layers[i](x)) # Apply CNN and ReLU activation

                # Apply max-pooling layers
                for layer in range(Config.cnn_pooling_layers):
                    x = self.pool(x)
            
            x = F.relu(x) # Apply ReLU activation to the output
            x_list.append(x) # Store the output in x_list

        # Reshape the embeddings before concatenation
        reshaped_x_list = []
        for i, x in enumerate(x_list):
            if isinstance(self.layers[i], (GCNConv, GATConv)):
                n_node = data_list[i].num_nodes // batch_size # Number of nodes per grap
                x = torch.reshape(x, (batch_size, n_node, x.shape[1])) # Reshape to (batch_size, nodes, features)
            reshaped_x_list.append(x)
        
        # Concatenating all inputs into a single tensor
        x = torch.cat(reshaped_x_list, dim=-1)

        # Reshape before passing into transformer
        if len(x.shape) == 3:
            # No need to permute if batch_first=True 
            pass
        elif len(x.shape) == 4:
            x = x.permute(0, 2, 1, 3)
            x = x.reshape(x.shape[0], x.shape[1], -1) # Flatten the last two dimensions

        # Pass through transformer and linear layers
        src, tgt = x, x # Using the same tensor for source and target in transformer
        x = self.transformer(src, tgt)
        x = torch.squeeze(x[:, -1, :])  # Squeeze to remove unnecessary dimensions after transformer
        x = self.linear(x) # Apply final linear transformation
        
        # Reshaping the output to match the original node structure
        total_nodes = src.shape[1] # No. of nodes
        s = x.shape
        # print (8, s)
        x = torch.reshape(x, (s[0], total_nodes, data_list[0].y.shape[1])) # Reshape to (batch_size, nodes, output_dim
        x = torch.reshape(x, (s[0] * total_nodes, data_list[0].y.shape[1])) # Flatten the batch and node dimensions

        return x

## MODEL TRAINING

In [11]:
def train_model(model, train_loader, optimizer, criterion):
    """
    Training function for the model.

    Parameters:
    - model: The DualGAT_Trans model to be trained.
    - train_loader: DataLoader for the training data.
    - optimizer: Optimizer for gradient descent.
    - criterion: Loss function to minimize.

    Returns:
    - total_loss: Average training loss over the epoch.
    """
    
    model.train() # Set the model to training mode
    total_loss = 0
    
    # Loop through the training batches
    for batch in zip(*train_loader):
        optimizer.zero_grad() # Zero the gradients
        out = model(batch) # Forward pass
        loss = criterion(out, batch[0].y) # Compute loss
        loss.backward() # Backpropagation
        optimizer.step() # Update weights
        total_loss += loss.item() # Accumulate loss
        
    return total_loss / len(train_loader[0]) # Return the average loss

## MODEL EVALUATION

In [12]:
def inv_transform(data, scaler, index_tf=2):
    """
    Inverse transform the output data using a scaler.

    Parameters:
    - data: Tensor of model outputs.
    - scaler: Scaler object used for inverse transformation.
    - index_tf: Index of the feature to inverse transform.

    Returns:
    - data: Tensor of inverse-transformed data, with negative values set to 0 and rounded to integers.
    """
    
    output_shape = data.shape
    data = data.detach().cpu().numpy().reshape(-1) # Convert tensor to NumPy array and flatten
    dummy_array = np.zeros((len(data), scaler.n_features_in_))  # Create a dummy array for inverse transform
    dummy_array[:, index_tf] = data # Place the data in the correct feature column
    
    # perform inverse transform
    inverse_transformed = scaler.inverse_transform(dummy_array)
    data = inverse_transformed[:, index_tf] # Extract the inverse-transformed data
    data = np.where(data < 0, 0, np.rint(data).astype(int)) # Set negatives to 0, round, and convert to int
    data = torch.tensor(data.reshape(output_shape), dtype=torch.float) # Convert back to tensor
    
    return data

In [13]:
def evaluate_model(model, test_loader, criterion):
    """
    Evaluate the model on the test data.

    Parameters:
    - model: The DualGAT_Trans model to be evaluated.
    - test_loader: DataLoader for the test data.
    - criterion: Loss function used for evaluation.

    Returns:
    - mae: Mean Absolute Error of predictions.
    - mape: Mean Absolute Percentage Error of predictions.
    - rmse: Root Mean Squared Error of predictions.
    - total_loss: Average loss over the test dataset.
    - predictions: Tensor of model predictions.
    - targets: Tensor of actual targets.
    """
    
    model.eval() # Set the model to evaluation mode
    total_loss = 0
    predictions, targets = [], []
    
    with torch.no_grad(): # Disable gradient computation
        for batch in zip(*test_loader):
            out = model(batch) # Forward pass
            loss = criterion(out, batch[0].y) # Compute loss
            total_loss += loss.item()  # Accumulate loss
            predictions.append(out) # Store predictions
            targets.append(batch[0].y) # Store actual targets
    
    # Concatenate all predictions and targets
    predictions = torch.cat(predictions, dim=0)
    targets = torch.cat(targets, dim=0)

    # Inverse transform the predictions and targets
    predictions = inv_transform(predictions, scaler, index_tf=2)
    targets = inv_transform(targets, scaler, index_tf=2)

    # Compute evaluation metrics
    mae = F.l1_loss(predictions, targets) # Mean Absolute Error
    rmse = torch.sqrt(F.mse_loss(predictions, targets)) # Root Mean Squared Error
    
    return mae, rmse, total_loss / len(test_loader[0]), predictions, targets

# STEP 4. RUN MODEL

In [14]:
def run_model(model, *train_data):
    """
    Train and evaluate the model on provided data.

    Parameters:
    - model: The DualGAT_Trans model to be trained and evaluated.
    - train_data: Tuple of datasets to be used for training and testing.

    Returns:
    - predictions: Final model predictions.
    - targets: Actual targets corresponding to the predictions.
    """
    
    train_loader, test_loader = [], []

    # Split data into training and testing sets, and create loaders
    for data in train_data:
        train_length = int(len(data) * Config.train_size) # Determine the split index
        train, test = data[:train_length].copy(), data[train_length:].copy() # Split the data
        train_loader.append(DataLoader(train, batch_size=Config.batch_size, shuffle=True))
        test_loader.append(DataLoader(test, batch_size=Config.batch_size, shuffle=False))

    # Training loop
    for epoch in tqdm(range(Config.epochs), desc="Training"):
        train_loss = train_model(model, train_loader, Config.optimizer, Config.criterion)
        
        if epoch % 5 == 0: # Evaluate every 5 epochs
            mae, rmse, test_loss, predictions, targets = evaluate_model(model, test_loader, Config.criterion)
            print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')
            print(f'MAE: {mae:.4f}, RMSE: {rmse:.4f}')
    
    # Final evaluation after all epochs
    mae, rmse, test_loss, predictions, targets = evaluate_model(model, test_loader, Config.criterion)
    print("Final Test Results:")
    print(f'MAE: {mae:.4f}, RMSE: {rmse:.4f}, Loss: {test_loss:.4f}')
    
    return predictions, targets

In [15]:
def run_models(model_designs, *inputs):
    """
    Train and evaluate multiple models based on different designs.

    Parameters:
    - model_designs: A dictionary where keys are model types and values are the corresponding parameters.
    - inputs: Tuple of datasets to be used for training and testing.

    Returns:
    - predictions_json: Dictionary containing predictions for each model type.
    - targets_json: Dictionary containing targets for each model type.
    """
    
    # Loop through each model type and run experiment
    predictions_json, targets_json = {}, {}
    for model_type, parameters in model_designs.items():
        print(f"Training {model_type} model")
        
        # Create model and optimizer
        model = DualGAT_Trans(model_params=parameters)

        # Set optimizer and loss function
        Config.optimizer = torch.optim.Adam(model.parameters(), lr=Config.learning_rate)
        Config.criterion = nn.MSELoss()

        # Train and evaluate the model
        predictions, targets = run_model(model, *inputs)

        # Store predictions and targets
        predictions_json[model_type] = predictions
        targets_json[model_type] = targets

    return predictions_json, targets_json

    

# VISUALISE RESULTS

In [16]:
def visualize_predictions(predictions, targets, title, x_label = 'Timestep'):
    """
    Visualize predictions against true values over time.

    Parameters:
    - predictions: Tensor of model predictions.
    - targets: Tensor of actual target values.
    - title: Title of the plot.
    - x_label: Label for the x-axis (default is 'Timestep').
    """

    plt.figure(figsize=(14, 3))

    # Average predictions and targets across the batch dimension
    predictions = torch.mean(predictions, axis=1)
    targets = torch.mean(targets, axis=1)

    # Plot true values and predictions
    plt.plot(targets, label='True Values', linewidth=1)
    plt.plot(predictions, label='Predictions', linewidth=1)
    plt.xlabel(x_label)
    plt.ylabel('Traffic Flow')
    plt.legend()
    plt.title(title)
    plt.show()

# DECISION SUPPORT SYSTEM: REINFORCEMENT LEARNING

## DATA CREATION

In [17]:
def create_rl_data(predictions, targets, meta_df, index_node_map):
    """
    Create a DataFrame suitable for Reinforcement Learning (RL) from model predictions and targets.

    Parameters:
    - predictions: Tensor of model predictions.
    - targets: Tensor of actual target values.
    - meta_df: DataFrame containing metadata about the nodes.
    - index_node_map: Mapping from index to node IDs.

    Returns:
    - rl_df: DataFrame with RL features and metadata.
    """
    
    rl_df = pd.DataFrame()

    # Reshape predictions and targets for processing
    targets_reshaped = targets.reshape(-1,len(index_node_map), targets.shape[1])
    predictions_reshaped = predictions.reshape(-1,len(index_node_map), predictions.shape[1])

    # Process each timestep: target and prediction data
    for i in range(targets_reshaped.shape[0]):
        
        dummy_df = pd.DataFrame(targets_reshaped[i], columns = ['target_' + str(i) for i in range(targets.shape[1])])
        dummy_df = dummy_df.reset_index()
        dummy_dfp = pd.DataFrame(predictions_reshaped[i], columns = ['pred_' + str(i) for i in range(predictions.shape[1])])
        dummy_dfp = dummy_dfp.reset_index()
        dummy_df['freeway_id'] = dummy_df['index'].map(index_node_map)
        dummy_df['timestep'] = i
        dummy_df = pd.concat([dummy_df, dummy_dfp], axis=1)
        rl_df = pd.concat([rl_df, dummy_df])

    # Merge with metadata
    rl_df = pd.merge(rl_df, meta_df, how='left', on = 'freeway_id')

    # Calculate average values
    rl_df['target_avg'] = np.mean(rl_df[['target_' + str(i) for i in range(targets.shape[1])]],axis=1)
    rl_df['target_avg_per_lane'] = rl_df['target_avg']/rl_df['lanes']
    rl_df['pred_avg'] = np.mean(rl_df[['pred_' + str(i) for i in range(predictions.shape[1])]],axis=1)
    rl_df['pred_avg_per_lane'] = rl_df['pred_avg']/rl_df['lanes']

    # Select relevant features
    rl_features = ['timestep','freeway_id','highway','freeway_direction','lanes','target_avg_per_lane','pred_avg_per_lane']
    rl_df = rl_df[rl_features]

    return rl_df


## OPPOSITE STATION MAPPING

In [18]:
def find_nearest_opposite_stations(df):
    """
    Find the nearest opposite direction stations for each station.

    Parameters:
    - df: DataFrame containing station metadata (latitude, longitude, freeway direction).

    Returns:
    - DataFrame with nearest opposite stations and distances.
    """
    # Group the dataframe by freeway_no
    grouped = df.groupby('highway')
    
    result = []
    
    # Function to add results to the list
    def add_into_result(nearest_indices, dir1, dir2, distances):
        for i, idx in enumerate(nearest_indices):
            result.append({
                    'station_id': dir1.iloc[i]['freeway_id'],#.name,
                    'nearest_station_id': dir2.iloc[idx]['freeway_id'],#.name,
                    'distance': distances[i, idx]
                })
        return 0
    
    for _, group in grouped:
        
        # Split the group into two based on freeway_direction
        dir1 = group[group['freeway_direction'] == group['freeway_direction'].iloc[0]]
        dir2 = group[group['freeway_direction'] != group['freeway_direction'].iloc[0]]
    
        
        if len(dir1) == 0 or len(dir2) == 0:
            continue
        
        # Calculate distances between all pairs of stations in opposite directions
        distances1 = cdist(dir1[['latitude', 'longitude']], dir2[['latitude', 'longitude']])
        distances2 = cdist(dir2[['latitude', 'longitude']], dir1[['latitude', 'longitude']])

        
        # Find the index of the nearest station for each station
        nearest_indices1 = np.argmin(distances1, axis=1)
        nearest_indices2 = np.argmin(distances2, axis=1)
        
        add_into_result(nearest_indices1, dir1, dir2, distances1)
        add_into_result(nearest_indices2, dir2, dir1, distances2)     
    
    return pd.DataFrame(result)

## MAP OPPOSITE STATIONS

In [19]:
def map_opp_stations(rl_df, opp_rl_df):
    """
    Map the nearest opposite stations to the RL DataFrame.

    Parameters:
    - rl_df: DataFrame with RL features and metadata.
    - opp_rl_df: DataFrame with nearest opposite stations.

    Returns:
    - rl_df: Updated DataFrame with nearest opposite station features.
    """
    
    rl_df = pd.merge(rl_df, opp_rl_df[['station_id','nearest_station_id']], how = 'left', left_on = 'freeway_id', right_on = 'station_id')
    
    # Define features to include
    rl_opp_features=['station_id','timestep','freeway_direction','lanes','target_avg_per_lane','pred_avg_per_lane']
    rl_df = pd.merge(rl_df, rl_df[rl_opp_features], how = 'left', left_on = ['nearest_station_id','timestep'], right_on = ['station_id','timestep'], suffixes = ("","_nearest_station")).drop(columns = ['freeway_id','nearest_station_id'])

    return rl_df

## GET STATE FROM TRAFFIC FLOW (REFERENCE & OPPOSITE STATION)

In [20]:
def get_state_index(state_space, input_values):
    """
    Get the index of the state in the state space that matches the given input values.

    Parameters:
    - state_space: Array of all possible states.
    - input_values: Values to find the nearest state index for.

    Returns:
    - index: Index of the matching state, or None if no match is found.
    """
    
    val1, val2 = input_values
    
    # Find the nearest lower or equal bin value for both elements
    bin1 = np.max(state_space[:, 0][state_space[:, 0] <= val1])
    bin2 = np.max(state_space[:, 1][state_space[:, 1] <= val2])
    
    # Find the index of the row in the state_space that matches the bin values
    index = np.where((state_space[:, 0] == bin1) & (state_space[:, 1] == bin2))[0]
    
    if len(index) > 0:
        return index[0]  # Return the first matching index
    else:
        return None

## GET RELEVANT STATES TO UPDATE REWARDS BASED ON CURRENT STATE

In [21]:
def get_related_state_indices(state_space, values, mode = 'reverse'):
    """
    Get indices of related states based on given values and mode.

    Parameters:
    - state_space: Array of all possible states.
    - values: Values to find related states for.
    - mode: Mode for finding related states ('reverse' or 'non-reverse').

    Returns:
    - indices: Indices of related states.
    """
    
    val1, val2 = values
    if mode == 'reverse':
        # Condition 1: 1st element >= val1 and 2nd element = val2
        cond1 = (state_space[:, 0] >= val1) & (state_space[:, 1] == val2)
        # Condition 2: 1st element = val1 and 2nd element <= val2
        cond2 = (state_space[:, 0] == val1) & (state_space[:, 1] <= val2)
    else:
        # Condition 1: 1st element >= val1 and 2nd element = val2
        cond1 = (state_space[:, 0] <= val1) & (state_space[:, 1] == val2)
        # Condition 2: 1st element = val1 and 2nd element <= val2
        cond2 = (state_space[:, 0] == val1) & (state_space[:, 1] >= val2)
    # Combine both conditions using logical OR
    indices = np.where(cond1 | cond2)[0]
    
    return indices

## LANE REVERSAL DECISION MODEL: REINFORCEMENT LEARNING

In [22]:
def rl_model(rl_df, n_episodes=250):
    """
    Train a Q-learning model for decision making based on RL data.

    Parameters:
    - rl_df: DataFrame with RL features.
    - n_episodes: Number of episodes for training.

    Returns:
    - Q: Q-table after training.
    - predict_action: Function to predict actions based on Q-table.
    - state_space: Array of all possible states.
    """
    
    # Define parameters
    n_actions = 2  # [0: No Reversal, 1: Reversal]
    learning_rate = 0.1
    discount_factor = 0.9
    epsilon = 0.7  # Exploration rate
    
    max_value = rl_df['pred_avg_per_lane'].max()

    # Create bins for state space
    bin_size = 20
    bins = np.arange(0, max_value + bin_size, bin_size)
    
    # Generate the cross product of the bins
    x, y = np.meshgrid(bins, bins)
    state_space = np.column_stack([x.ravel(), y.ravel()])

    # Initialize Q-table
    Q = np.zeros((len(state_space), n_actions))
    
    
    # Reward function
    def get_reward(pred_current, pred_opposite, actual_current, actual_opposite, action):
        if action == 1:  # Reversal
            if actual_current/max(1, actual_opposite) > 2 and actual_current>90: # Positive reward for successful decision
                if pred_current/max(1, pred_opposite) > 2 and pred_current>90:
                    return 20  
                else:
                    return 0
            else:
                if pred_current/max(1, pred_opposite) > 2 and pred_current>90: 
                    return 10  # Positive reward for semi-successful decision
                else:
                    return -10  # Negative reward for unsuccessful decision
        else:  # No reversal
            if not (actual_current/max(1, actual_opposite) > 2 and actual_current>90): # Positive reward for successful decision
                if not (pred_current/max(1, pred_opposite) > 2 and pred_current>90):
                    return 10  
                else:
                    return 5
            else:  # Negative reward for unsuccessful decision
                if not(pred_current/max(1, pred_opposite) > 2 and pred_current>90):
                    return -5  
                else:
                    return -10 #
    
    # Q-learning algorithm
    for episode in tqdm(range(n_episodes), leave=False):
        for _, row in rl_df.iterrows():
            row['pred_avg_per_lane'] = min(row['pred_avg_per_lane'], rl_df['target_avg_per_lane'].max())
            row['pred_avg_per_lane_nearest_station'] = min(row['pred_avg_per_lane_nearest_station'], rl_df['target_avg_per_lane'].max())
            state = get_state_index(state_space, (row['pred_avg_per_lane'],row['pred_avg_per_lane_nearest_station']))
            action = np.random.randint(n_actions) if np.random.rand() < epsilon else np.argmax(Q[state, :])
            
            # Get reward based on actual values for verification
            reward = get_reward(
                row['pred_avg_per_lane'], 
                row['pred_avg_per_lane_nearest_station'], 
                row['target_avg_per_lane'], 
                row['target_avg_per_lane_nearest_station'], 
                action
            )
            
            # Update Q-value
            if action==1:
                state_to_update = get_related_state_indices(state_space, state_space[state], mode = 'reverse')
            else:
                state_to_update = get_related_state_indices(state_space, state_space[state], mode = 'non-reverse')

            # For simplicity, assume the next state is the same as the current state
            next_state = state
            best_next_action = np.argmax(Q[next_state, :])
            Q[state_to_update, action] += learning_rate * (reward + discount_factor * Q[next_state, best_next_action] - Q[state_to_update, action])
    
    # Function to predict action based on Q-table
    def predict_action(pred_current, pred_opposite):
        state = get_state_index(state_space, (pred_current,pred_opposite))
        return np.argmax(Q[state, :])

    return Q, predict_action, state_space

## LANE REVERSAL MODEL EVALUATION

In [23]:
def dss_result_plot(rl_df):
    """
    Plot results of the decision support system (DSS) with lane reversal flag.

    Parameters:
    - rl_df: DataFrame with RL features including lane reversal.
    """
    
    plt.figure(figsize=(10, 6))
    
    # Plot with different colors based on 'lane_reversal' flag
    colors = {1: 'red', 0: 'green'}
    plt.scatter(rl_df['pred_avg_per_lane_nearest_station'], rl_df['pred_avg_per_lane'],
                c=rl_df['lane_reversal'].map(colors),
                label=rl_df['lane_reversal'])
    
    # Adding labels and title
    plt.ylabel('Reference Station Traffic')
    plt.xlabel('Opp Station Traffic')
    plt.title('Scatter Plot of Traffic with Lane Reversal Flag')
    
    # Creating a legend
    handles = [plt.Line2D([0], [0], marker='o', color='w', label='Reversal',
                          markerfacecolor='red', markersize=10),
               plt.Line2D([0], [0], marker='o', color='w', label='No Reversal',
                          markerfacecolor='green', markersize=10)]
    plt.legend(handles=handles)
    
    # Show plot
    plt.show()